## XGBoost


xgboost是大规模并行boosted tree的工具，它是目前最快最好的开源boosted tree工具包，比常见的工具包快10倍以上。在数据科学方面，有大量kaggle选手选用它进行数据挖掘比赛，其中包括两个以上kaggle比赛的夺冠方案。在工业界规模方面，xgboost的分布式版本有广泛的可移植性，支持在YARN, MPI, Sungrid Engine等各个平台上面运行，并且保留了单机并行版本的各种优化，使得它可以很好地解决于工业界规模的问题。

---

### 1.回顾监督学习知识点

- $x_i \in R^d$代表第$i$个训练样本
- 模型：给定$x_i$作出预测$\hat{y_i}$
    - 线性模型：$\hat{y_i} = \sum_j w_j x_{ij}$，包括线性回归和逻辑回归
    - 基于不同的学习任务，$\hat{y_i}$有不同的解释
        - 线性回归：$\hat{y_i}$是预测的值
        - 逻辑回归：$\frac{1}{1+exp(-\hat{y_i})}$可以看成是预测为正类的概率
        - 其它的模型中，比如排名的得分
- 参数：需要从数据集中训练出来的
    - 线性模型：$$\theta =\{w_j \mid j=1, ..., d\}$$
    
- 目标函数：$$Obj(\theta) = L(\theta) + \Omega(\theta)$$
    - 第一项是训练误差，评测模型的训练集的拟合能力
    - 第二项正则，评测模型的复杂性
- 训练误差：$$L = \sum_{i=1}^{n} l(y_i, \hat{y_i})$$
    - 平方误差：$$l(y_i, \hat{y_i}) = (y_i - \hat{y_i})^2$$
    - 对数误差：$$l(y_i, \hat{y_i}) = y_i \ln (1+e^{-y_i}) + (1-y_i) \ln (1+ e^{\hat{y_i}})$$
- 正则化：
    - L2正则：$$\Omega(w) = \lambda \|w\|^2$$
    - L1正则：$$\Omega(w) = \lambda \|w\|_1$$
- 基于以上内容的汇总：
    - Ridge回归（线性模型，平方误差，L2正则）：$$\sum_{i=1}^{n}(y_i - w^T x_i)^2 + \lambda \|w\|^2$$
    - Lasso回归（线性模型，平方误差，L1正则）：$$\sum_{i=1}^{n}(y_i - w^T x_i)^2 + \lambda \|w\|_1$$
    - 逻辑回归（线性模型，对数损失，L2正则）：$$\sum_{i=1}^{n}[y_i \ln (1+e^{-y_i}) + (1-y_i) \ln (1+ e^{\hat{y_i}})] + \lambda \|w\|^2$$
    
- 偏差-方差平衡
    - 优化训练误差，可以得到一个更接近训练集数据特性的模型
    - 优化正则项，使得模型更简单，在测试集上的预测具有更小的方差，预测效果更稳定

---

### 2.回归树

- 分类回归树 CART: classification and regression tree
    - 判决规则和决策树类似
    - 既可用于回归，又可用于分类
    - 每个叶子结点都带有一个分值
        - 例子：根据年龄、性别、职业等特征，预测一个人是不是游戏玩家
        ![](tree_example.png)

- 回归树集成
    - 对样本用多棵树进行预测，预测结果为每棵树预测分数的和
        - 还是上面的例子，增加一棵树，比如根据使用电脑频率的特征来进行判定
        ![](tree_ensemble_example.png)
    
    - 随机森林是树集成一个典型的应用

    - 模型
       - 假设有k棵树，最终的预测值为：$$\hat{y_i} = \sum_{k=1}^{K}f_k(x_i)$$
    - 目标函数：$$Obj = \sum_{i=1}^{n} l(y_i, \hat{y_i}) + \sum_{k=1}^{K} \Omega(f_k)$$
        - 第一项代表训练误差，第二项代表树的复杂程度
        - 可能用来定义树复杂程度的方式
            - 每棵树的叶子结点数，树的深度
            - 对叶子结点权重的L2正则
            - 其它
        - 决策树通常是启发式构建的，或者说贪心的：
            - 信息增益->选择划分特征
            - 树的剪枝->非叶子结点数的正则
            - 最大化树的深度->在函数空间进行限制
            - 叶子结点得分的平滑处理－>对叶子结点权重的L2正则
            
    - 不能像线性回归一样，使用随机梯度下降(SGD)方法对目标函数进行优化，因为这些是树模型，不仅仅是矩阵计算
    - 累积计算
        - Additive Training(Boost)
        - 从常量开始预测，每次都添加一个新的函数：
        $$\hat{y_i}^{(0)} = 0 
        \\ \hat{y_i}^{(1)} = f_1(x_i) = \hat{y_i}^{(0)} + f_1(x_i)
        \\ \hat{y_i}^{(2)} = f_1(x_i) + f_2(x_i) = \hat{y_i}^{(1)} + f_2(x_i)
        \\ ...
        \\ \hat{y_i}^{(t)} = \sum_{k=1}^{K}f_k(x_i) = \hat{y_i}^{(t-1)} + f_t(x_i)$$
        
        - 经过t棵树的预测：$$\hat{y_i}^{(t)} = \hat{y_i}^{(t-1)} + f_t(x_i)$$
        - 目标函数为：$$Obj^{(t)} = \sum_{i=1}^{n} l(y_i, \hat{y_i}^{(t)}) + \sum_{i=1}^{t}\Omega(f_i)
          \\ = \sum_{i=1}^{n} l(y_i, \hat{y_i}^{(t-1)} + f_t(x_i) ) + \Omega(f_t) + constant $$
          
        - 对目标函数进行多元泰勒展开：
            - 多元泰勒展开：$f(x+\Delta x, y+ \Delta y) = f(x,y) + (\Delta x \frac{\partial}{\partial x} + \Delta y \frac{\partial}{\partial y}) f(x,y) + \frac{1}{2!}((\Delta x)\frac{\partial}{\partial x} + \Delta y \frac{\partial}{\partial y})^2 f(x,y) + ...$
            
            - 定义：
                - $\Delta x = 0$，$\Delta y = f_t(x_i)$
                - $ g_i =  \partial_{\hat{y}^{(t-1)}} l(y_i, \hat{y}^{(t-1)}) $，$h_i =  \partial_{\hat{y}^{(t-1)}}^2 l(y_i, \hat{y}^{(t-1)})$
                
            - 目标函数近似：
                $$Obj^{(t)} \approx \sum_{i=1}^{n} [l(y_i, \hat{y_i}^{(t-1)}) + g_i f_t(x_i) + \frac{1}{2}h_i f_t^2(x_i)] + \Omega (f_t) + constant$$
                
            - 去掉常数项之后，就会得到一个比较统一的目标函数，并且有一个非常明显的特点：**只依赖于每个数据点在误差函数上的一阶导数和二阶导数**：$$\sum_{i=1}^{n} [g_i f_t(x_i) + \frac{1}{2}h_i f_t^2(x_i)] + \Omega (f_t)$$
            
            - 推导的目的
                - 理论上：有助于我们了解这个模型进行任务学习的过程
                - 工程上：根据回归和分类，选择相应的损失函数，使得这个理论更具普遍性
                
        - 以上是对目标函数中训练误差的讨论，下面讨论如何定义树的复杂度
        
        - 对树f()进行重定义，分为树的结构部分和叶子结点的权重构成的向量，如下图所示：
        ![](tree_refine.png)
        
        - 因此将树的复杂度定义为：$$\Omega (f_t) = \gamma T + \frac{1}{2} \lambda \sum_{j=1}^{T} w_j^2$$
            - 其中，T代表叶子的结点数，第二项是对叶子结点分数的L2正则
            - 如下图所示：
            ![](tree_regulization.png)
            
        - 基于以上信息，对目标函数进行总结：
            - 定义：每个叶子结点上样本的集合$I_j = \{i \mid q(x_i) = j\}$
            - 目标函数：
                $$Obj^{(t)} \approx \sum_{i=1}^{n} [g_i f_t(x_i) + \frac{1}{2}h_i f_t^2(x_i)] + \Omega (f_t) 
                \\ = \sum_{i=1}^{n} [g_i f_t(x_i) + \frac{1}{2}h_i f_t^2(x_i)] + \gamma T + \frac{1}{2} \lambda \sum_{j=1}^{T} w_j^2
                \\ = \sum_{j=1}^{T} [(\sum_{i \in I_j} g_i) w_j + \frac{1}{2}(\sum_{i \in I_j} h_j + \lambda)w_j^2] + \gamma T$$
            
            - 定义：$$G_j = \sum_{i \in I_j} g_i \quad H_j = \sum_{i \in I_j} h_i$$
            - 目标函数可以重写为：$$Obj^{(t)} = \sum_{j=1}^{T}[G_j w_j + \frac{1}{2}(H_j + \lambda)w_j^2] + \gamma T$$
            - 目标函数的最小值，也就转化为一元二次函数的最小值问题
            - 一元二次方程两个结论：
            $$\arg \min_x Gx + \frac{1}{2}Hx^2 = -\frac{G}{H}, H >0 \quad \min_x Gx + \frac{1}{2}Hx^2 = -\frac{1}{2} \frac{G^2}{H}$$
                - 这句话的意思是说，对$Gx + \frac{1}{2}Hx^2 = x(G+\frac{1}{2}Hx)$取最小值时，$x=-\frac{G}{H}$，并且此时的最小值为$-\frac{1}{2} \frac{G^2}{H}$
            
            - 根据上面一元二次方程的结论，在每个叶子结点的权重值为：$$w_j^* = - \frac{G_j}{H_j+\lambda}$$
            - 目标函数的最小值为：$$-\frac{1}{2}\sum_{j=1}^{T}\frac{G_j^2}{H_j + \lambda} + \gamma T$$
                - 只需计算每个样本的一次偏导和二次偏导即可
                - 值越小，树的结构更优
                ![](score_calculation.png)
            
            - 不断地枚举所有可能的树的结构，并计算相应目标函数的最小值，可以得出一个最优结构的树。
            - 简单的枚举计算量比较大，常用的方法是贪心法.
            - 每次尝试去对已有的叶子根据某个特征加入一个分割，计算分割后的增益，选择增益最大的作为分割条件
            $$Gain = \frac{1}{2}[\frac{G_L^2}{H_L + \lambda} + \frac{G_R^2}{H_R + \lambda} - \frac{(G_L+ G_R)^2}{H_L + H_R+ \lambda}] - \gamma$$
                - $\frac{G_L^2}{H_L + \lambda}$，$\frac{G_R^2}{H_R + \lambda}$分割后左子树和右子树的分数
                - $\frac{(G_L+ G_R)^2}{H_L + H_R+ \lambda}$，分割前结点的分数
                - 对叶子结点进行分割，和分割前相比多了一个叶子结点，引入了一个$\gamma$
            - 引入分割不一定会使得情况变好，因为有引入一个新叶子的惩罚项。优化这个目标相当于对树的剪枝，当引入的分割带来的增益小于一个阈值的时候，可以剪掉这个分割。所以在进行推导的时候，像计算分数和剪枝这样的策略都会自然地出现。
         
     - xgboost工具支持并行。boosting不是一种串行的结构吗?怎么并行的？注意xgboost的并行不是tree粒度的并行，xgboost也是一次迭代完才能进行下一次迭代的（第t次迭代的代价函数里包含了前面t-1次迭代的预测值）。xgboost的并行是在特征粒度上的。我们知道，决策树的学习最耗时的一个步骤就是对特征的值进行排序（因为要确定最佳分割点），xgboost在训练之前，预先对数据进行了排序，然后保存为block结构，后面的迭代中重复地使用这个结构，大大减小计算量。这个block结构也使得并行成为了可能，在进行节点的分裂时，需要计算每个特征的增益，最终选增益最大的那个特征去做分裂，那么各个特征的增益计算就可以开多线程进行。
    ![](xgboost_parallel.png)



 ### 总结
 - 1.需要计算目标函数的一阶导数和二阶导数
 - 2.基于贪心法获得最优切分点
 - 3.支持特征粒度的并行计算
 - 4.对于线性分类器自带L1和L2正则
 - 5.自带剪枝